In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from math import sqrt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam


In [2]:
# Membaca file CSV
df = pd.read_csv('data_harga_beras.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date').interpolate(method='linear').reset_index()

In [3]:
# Drop kolom yang tidak digunakan
df = df.drop(['C4Super', 'Bulog'], axis=1)
df = df.filter(['C4Medium'])
df = df.values.astype('float32')

In [4]:
# Normalisasi data dengan range [0, 1] sebelum membagi data
scaler_0_1 = MinMaxScaler(feature_range=(0, 1))
df_normalized_0_1 = scaler_0_1.fit_transform(df)

scaler_neg1_1 = MinMaxScaler(feature_range=(-1, 1))
df_normalized_neg1_1 = scaler_neg1_1.fit_transform(df)

In [5]:
# Fungsi untuk menyiapkan data
def prepare_data(df, timesteps):
    X, Y = [], []
    for i in range(len(df) - timesteps - 1):
        X.append(df[i:(i + timesteps)])
        Y.append(df[i + timesteps, 0])
    return np.array(X), np.array(Y)

In [6]:
# Fungsi untuk membangun dan melatih model
def build_and_train_model(df_normalized, timesteps=1, num_neuron=10, num_epoch=50, num_batch=16):
    # Split data menjadi training dan testing
    n_samples = len(df_normalized)
    train_size = int(n_samples * 0.7)
    test_size = n_samples - train_size

    train_data = df_normalized[:train_size]
    test_data = df_normalized[train_size:]
    
    # Persiapan data
    X_train, Y_train = prepare_data(train_data, timesteps)
    X_test, Y_test = prepare_data(test_data, timesteps)

    # Reshape data
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

    # Build model
    model = Sequential()
    model.add(LSTM(4, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(num_neuron, activation='tanh', recurrent_activation='sigmoid', return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1, bias_initializer='zeros'))
    
    # Compile model
    adam_optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=adam_optimizer, loss='mse', metrics=['MAPE'])
    
    # Train model
    history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=num_epoch, batch_size=num_batch, verbose=1)
    
    return model, X_test, Y_test, history


In [7]:
# Normalisasi dengan range [0, 1]
model_0_1, X_test_0_1, Y_test_0_1, history_0_1 = build_and_train_model(df_normalized_0_1)

# Normalisasi dengan range [-1, 1]
model_neg1_1, X_test_neg1_1, Y_test_neg1_1, history_neg1_1 = build_and_train_model(df_normalized_neg1_1)

Epoch 1/50


C:\Users\Windows\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


53/53 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - MAPE: 470679.5938 - loss: 0.0155 - val_MAPE: 74.4401 - val_loss: 0.2603
Epoch 2/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 1963864.2500 - loss: 0.0089 - val_MAPE: 68.9319 - val_loss: 0.2299
Epoch 3/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 1878749.5000 - loss: 0.0085 - val_MAPE: 65.3256 - val_loss: 0.2069
Epoch 4/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 2389088.7500 - loss: 0.0083 - val_MAPE: 61.9013 - val_loss: 0.1831
Epoch 5/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 1580164.8750 - loss: 0.0076 - val_MAPE: 55.3427 - val_loss: 0.1449
Epoch 6/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 1128604.5000 - loss: 0.0050 - val_MAPE: 44.9085 - val_loss: 0.0939
Epoch 7/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 2332826.7500 - loss: 0.0039 - val_MAPE: 31.1087 - val_loss: 0.0424
Epoch 8/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 750942.7500 - loss: 0.0032 - val_MAPE: 16.6704 - val_loss: 0.0108
Epoch 9/50
5

53/53 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - MAPE: 94.1768 - loss: 0.5566 - val_MAPE: 129.3926 - val_loss: 0.2712
Epoch 2/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 72.5181 - loss: 0.3615 - val_MAPE: 183.8703 - val_loss: 0.3372
Epoch 3/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 50.2557 - loss: 0.1694 - val_MAPE: 272.5170 - val_loss: 0.4422
Epoch 4/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 31.4521 - loss: 0.0517 - val_MAPE: 319.3645 - val_loss: 0.5030
Epoch 5/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 34.0811 - loss: 0.0410 - val_MAPE: 316.4313 - val_loss: 0.4975
Epoch 6/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 29.6438 - loss: 0.0387 - val_MAPE: 313.6634 - val_loss: 0.4926
Epoch 7/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 31.4544 - loss: 0.0383 - val_MAPE: 307.5064 - val_loss: 0.4823
Epoch 8/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 29.9471 - loss: 0.0364 - val_MAPE: 306.6732 - val_loss: 0.4790
Epoch 9/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3

In [8]:
# Evaluasi model sebelum denormalisasi
def evaluate_model_before_denormalization(model, X_test, Y_test):
    # Predict
    test_predict = model.predict(X_test, verbose=0)

    # Calculate metrics
    mae = mean_absolute_error(Y_test, test_predict)
    rmse = sqrt(mean_squared_error(Y_test, test_predict))
    mape = mean_absolute_percentage_error(Y_test, test_predict) * 100

    return mae, rmse, mape, Y_test, test_predict

In [9]:

# Evaluasi model untuk kedua skala
mae_0_1, rmse_0_1, mape_0_1, Y_test_0_1, test_predict_0_1 = evaluate_model_before_denormalization(model_0_1, X_test_0_1, Y_test_0_1)
mae_neg1_1, rmse_neg1_1, mape_neg1_1, Y_test_neg1_1, test_predict_neg1_1 = evaluate_model_before_denormalization(model_neg1_1, X_test_neg1_1, Y_test_neg1_1)

# Print hasil evaluasi
print(f"Interval [0, 1] -> RMSE: {rmse_0_1}, MAPE: {mape_0_1}%")
print(f"Interval [-1, 1] -> RMSE: {rmse_neg1_1}, MAPE: {mape_neg1_1}%")


Interval [0, 1] -> RMSE: 0.041299962603000645, MAPE: 5.016615241765976%
Interval [-1, 1] -> RMSE: 0.4773514267823907, MAPE: 136.69270277023315%
